There exist mentions of elibility requirements in page text, some in specialized `Eligibility` sections, others within the context of `smart_answers` or `answer` document_types. With a focus on age requirements, extract stuff out.

In [1]:
import os
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from pprint import pprint

In [2]:
DATA_DIR = os.getenv("DATA_DIR")
content_path = os.path.join(DATA_DIR, 
                            "preprocessed_content_store_wdetails_june_en_june.csv.gz")
df = pd.read_csv(content_path, compression="gzip")

/Users/felisialoukou/.virtualenvs/k-graph/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale
0,/aaib-reports/aaib-investigation-to-hawker-sea...,96eacfbe-0385-45ef-9289-8428dacad258,"AAIB investigation to Hawker Sea Fury T MK 20,...","Engine failure and landing gear collapse, RNAS...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The aircraft was performing in a publ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
1,/aaib-reports/aaib-investigation-to-hph-glasfl...,1d697c99-b1d0-4855-b72d-a97d83a4fc91,"AAIB investigation to HPH Glasflugel 304 eS, G...",Front Electric Sustainer (FES) battery fire du...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: During a normal touchdown following a...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
2,/aaib-reports/aaib-investigation-to-ikarus-c42...,5814334a-77d0-426e-8e78-ff2b05ea6322,"AAIB investigation to Ikarus C42 FB UK, G-IKUS\t",Aircraft crashed whilst avoiding a hedge when ...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot was attempting to take off ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
3,/aaib-reports/aaib-investigation-to-ikarus-c42...,c8f31c76-eab0-4be6-95a5-5e6e7f32056c,"AAIB investigation to Ikarus C42 FB100, G-CEHG\t","Overturned on landing, Farm Strip, Hardwicke, ...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: G-CEHG was landing on a private airst...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
4,/aaib-reports/aaib-investigation-to-ikarus-c42...,4cc0ae15-ad87-42ce-8a89-2bdb14e19e26,"AAIB investigation to Ikarus C42 FB100, G-ZAVI","Aircraft struck sheep on landing, Lundy Island...",aaib_report,{'metadata': {'date_of_occurrence': '2014-06-2...,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot reported that this was his ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en


In [4]:
df['details'] = df['details'].map(literal_eval)

In [5]:
### this condition on a dictionary element is very useful
parts_test = df[df.details.apply(lambda x: "parts" in x.keys())].iloc[0].details['parts']
parts_test

[{'title': 'Overview',
  'slug': 'overview',
  'body': [{'content_type': 'text/govspeak',
    'content': "You could get a bursary to help with education-related costs if you’re aged 16 to 19 and: \r\n\r\n+ studying at a publicly funded school or college in England - not a university  \r\n+ on a training course, including unpaid work experience  \r\n\r\nA publicly funded school is one that does not charge you for attending it.\r\n\r\n^There's a different [scheme in Wales, Scotland and Northern Ireland](/education-maintenance-allowance-ema).^ \r\n\r\n##If you're 19 and over\r\n\r\nYou could also get a bursary if you either:  \r\n\r\n* are continuing on a course you started aged 16 to 18 (known as being a ’19+ continuer’)  \r\n* have an [Education, Health and Care Plan (EHCP)](/children-with-special-educational-needs/extra-SEN-help)  \r\n\r\n##What a bursary is for\r\nA bursary is money that you, or your education or training provider, can use to pay for things like: \r\n\r\n+ clothing, b

In [6]:
def flatten_dict(parts_dict):
    return {item['slug']:"".join([b['content'] for b in item['body']]) for item in parts_dict}    

In [7]:
flatten_dict(parts_test)['overview'][0:100]

'You could get a bursary to help with education-related costs if you’re aged 16 to 19 and: \r\n\r\n+ stud'

In [8]:
df['details_parts'] = df['details'].map(lambda x: flatten_dict(x['parts']) 
                                        if 'parts' in x.keys() 
                                          else np.nan)

In [9]:
df['eligibility_html'] = df['details_parts'].map(lambda x: x.get('eligibility',np.nan) 
                                                 if not isinstance(x,float) else np.nan)

In [10]:
df_welig = df[~df['eligibility_html'].isna()]

In [11]:
elig_texts = dict(zip(df_welig.base_path, df_welig.eligibility_html))

In [12]:
elig_urls = [f'https://www.gov.uk{bp}/eligibility' for bp in df_welig.base_path.values]

In [13]:
for u in elig_urls[0:2]:
    print(u)

https://www.gov.uk/1619-bursary-fund/eligibility
https://www.gov.uk/additional-state-pension/eligibility


In [14]:
rowlist = []
for key,value in list(elig_texts.items()):

    if "This fund is currently closed." not in value:
#         \s?\d\d\s?(and|or)?\s?(over|under)?(\s\d\d)?
        search = re.finditer(""".*(born on|(reached|over|under) State Pension age|
                             age(d)?|('re|were|are|be( at least)?|have been)\s?(over|under)?\d\d).*""", value)
        if search is not None:
#             print(f'https://www.gov.uk{key}/eligibility')
            for i,s in enumerate(search):
                start = s.start()
                end = s.end()
#                 print(f'At {i}: ({start},{end})')
                text = value[start:end]
#                 print(text)
                rowlist.append({'base_path':key, 
                                'instance #':int(i+1),
                                'start':start,
                                'end':end,
                                'age_text':value[start:end],
                                'text':value})
        else:
            rowlist.append({'base_path':key, 'age_text':'Age requirements not detected',
                           'text':value})
            
    else:
#         print(f'https://www.gov.uk{key}/eligibility')
#         print("closed")
        rowlist.append({'base_path':key, 'age_text':'Fund closed','text':value})
df_eligibility = pd.DataFrame(rowlist)
df_eligibility = df_eligibility[['base_path', 'text','instance #', 'start', 'end','age_text']]

In [15]:
base_title = dict(zip(df_welig.base_path,df_welig.title))
base_taxon = dict(zip(df_welig.base_path,df_welig.taxons))

In [16]:
df_eligibility['title'] = df_eligibility['base_path'].map(base_title)
df_eligibility['taxon'] = df_eligibility['base_path'].map(base_taxon)
df_eligibility['base_path'] = df_eligibility['base_path'].map(lambda x: 
                                                              f'https://www.gov.uk{x}/eligibility')

In [24]:
df_eligibility['taxon'] = df_eligibility['taxon'].map(lambda x: literal_eval(x) if not isinstance(x,float) else [])

In [25]:
df_eligibility['taxon'] = df_eligibility['taxon'].map(lambda x: [xs['title'] for xs in 
                                                                 x])

In [26]:
df_eligibility

,base_path,text,instance #,start,end,age_text,title,taxon
0,https://www.gov.uk/1619-bursary-fund/eligibility,You must:\r\n\r\n+ be at least 16 and under 19...,1.0,13.0,61.0,+ be at least 16 and under 19 on 31 August 2019\r,16 to 19 Bursary Fund,"[Education, training and skills]"
1,https://www.gov.uk/1619-bursary-fund/eligibility,You must:\r\n\r\n+ be at least 16 and under 19...,2.0,1772.0,1828.0,<li>be at least 16 and under 19 on 31 August...,16 to 19 Bursary Fund,"[Education, training and skills]"
2,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,1.0,0.0,57.0,##You reached State Pension age on or after 6 ...,Additional State Pension,"[Work, Money]"
3,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,2.0,250.0,303.0,##You reached State Pension age before 6 April...,Additional State Pension,"[Work, Money]"
4,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,3.0,306.0,523.0,If you reached State Pension age before 6 Apri...,Additional State Pension,"[Work, Money]"
5,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,4.0,1151.0,1271.0,12 October 2015 to 5 April 2017 | State Pensio...,Additional State Pension,"[Work, Money]"
6,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,5.0,1895.0,2018.0,"<h2 id=""you-reached-state-pension-age-on-or-af...",Additional State Pension,"[Work, Money]"
7,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,6.0,2236.0,2349.0,"<h2 id=""you-reached-state-pension-age-before-6...",Additional State Pension,"[Work, Money]"
8,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,7.0,2351.0,2574.0,<p>If you reached State Pension age before 6 A...,Additional State Pension,"[Work, Money]"
9,https://www.gov.uk/additional-state-pension/el...,##You reached State Pension age on or after 6 ...,8.0,3666.0,3743.0,<td>You reached State Pension age before...,Additional State Pension,"[Work, Money]"


In [27]:
df_eligibility = df_eligibility[['base_path', 'title',
                                 'text','instance #', 'start', 'end','age_text','taxon']]

In [39]:
df_eligibility.taxon.iloc[0][0]

'Education, training and skills'

In [40]:
mask = [('Education, training and skills' in x) for x in df_eligibility['taxon']]
df_eligibility_ed = df_eligibility[mask]

In [41]:
df_eligibility_ed

,base_path,title,text,instance #,start,end,age_text,taxon
0,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,1.0,13.0,61.0,+ be at least 16 and under 19 on 31 August 2019\r,"[Education, training and skills]"
1,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,2.0,1772.0,1828.0,<li>be at least 16 and under 19 on 31 August...,"[Education, training and skills]"
23,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,1.0,204.0,268.0,You must have been 24 or older on the first da...,"[Education, training and skills]"
24,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,2.0,597.0,654.0,You must be 19 or older on the first day of yo...,"[Education, training and skills]"
25,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,3.0,2477.0,2547.0,<p>You must have been 24 or older on the first...,"[Education, training and skills]"
26,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,4.0,3067.0,3130.0,<p>You must be 19 or older on the first day of...,"[Education, training and skills]"
53,https://www.gov.uk/learner-support/eligibility,Learner Support,To get Learner Support you must be:\r\n\r\n- 1...,1.0,166.0,290.0,^You must be 20 or over to get help with child...,"[Education, training and skills]"
54,https://www.gov.uk/learner-support/eligibility,Learner Support,To get Learner Support you must be:\r\n\r\n- 1...,2.0,891.0,1030.0,<p>You must be 20 or over to get help with chi...,"[Education, training and skills]"


In [43]:
eligibility_path = os.path.join(DATA_DIR, "eligibility_guides_text_education.csv")
df_eligibility_ed.drop('taxon',axis=1,inplace=True)
df_eligibility_ed.to_csv(eligibility_path, index=False)

/Users/felisialoukou/.virtualenvs/k-graph/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [44]:
df_eligibility_ed

,base_path,title,text,instance #,start,end,age_text
0,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,1.0,13.0,61.0,+ be at least 16 and under 19 on 31 August 2019\r
1,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,2.0,1772.0,1828.0,<li>be at least 16 and under 19 on 31 August...
23,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,1.0,204.0,268.0,You must have been 24 or older on the first da...
24,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,2.0,597.0,654.0,You must be 19 or older on the first day of yo...
25,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,3.0,2477.0,2547.0,<p>You must have been 24 or older on the first...
26,https://www.gov.uk/advanced-learner-loan/eligi...,Advanced Learner Loan,Whether you qualify for an Advanced Learner Lo...,4.0,3067.0,3130.0,<p>You must be 19 or older on the first day of...
53,https://www.gov.uk/learner-support/eligibility,Learner Support,To get Learner Support you must be:\r\n\r\n- 1...,1.0,166.0,290.0,^You must be 20 or over to get help with child...
54,https://www.gov.uk/learner-support/eligibility,Learner Support,To get Learner Support you must be:\r\n\r\n- 1...,2.0,891.0,1030.0,<p>You must be 20 or over to get help with chi...
